In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Configure the plotting routines
import numpy as np
import pandas as pd
import base64

# Import your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Initialize the database connection with credentials
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# Load all data initially
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the MongoDB '_id' column to prevent data table crashes
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Debug information
print(f"Data loaded successfully: {len(df)} records")
print(f"Columns: {list(df.columns)}")

# Helper function to get rescue-specific data
def get_rescue_data(rescue_type):
    """
    Get filtered data based on rescue type using the criteria from the specifications
    """
    if rescue_type == "Water Rescue":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", 
                             "Newfoundland", "Portuguese Water Dog"]},
            "sex_upon_outcome": {"$regex": "Intact Female", "$options": "i"},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == "Mountain/Wilderness Rescue":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                             "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": {"$regex": "Intact Male", "$options": "i"},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == "Disaster/Individual Tracking":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                             "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": {"$regex": "Intact Male", "$options": "i"},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:  # Reset - return all data
        query = {}
    
    # Get filtered data from database
    filtered_records = shelter.read(query)
    filtered_df = pd.DataFrame.from_records(filtered_records)
    
    # Remove '_id' column if present
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    
    return filtered_df

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('GraziosoSalvareDashboard')

# Load the Grazioso Salvare logo from local file
try:
    with open('Grazioso Salvare Logo.png', 'rb') as f:
        encoded_image = base64.b64encode(f.read()).decode()
    logo_src = f"data:image/png;base64,{encoded_image}"
    print("Grazioso Salvare logo loaded successfully!")
except FileNotFoundError:
    print("Logo file not found. Using placeholder.")
    logo_src = "data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMTAwIiBoZWlnaHQ9IjEwMCIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj48cmVjdCB3aWR0aD0iMTAwIiBoZWlnaHQ9IjEwMCIgZmlsbD0iIzJFOEI1NyIvPjx0ZXh0IHg9IjUwIiB5PSI1NSIgZm9udC1mYW1pbHk9IkFyaWFsIiBmb250LXNpemU9IjE2IiBmaWxsPSJ3aGl0ZSIgdGV4dC1hbmNob3I9Im1pZGRsZSI+R1M8L3RleHQ+PC9zdmc+"
except Exception as e:
    print(f"Error loading logo: {e}")
    logo_src = "data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMTAwIiBoZWlnaHQ9IjEwMCIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj48cmVjdCB3aWR0aD0iMTAwIiBoZWlnaHQ9IjEwMCIgZmlsbD0iIzJFOEI1NyIvPjx0ZXh0IHg9IjUwIiB5PSI1NSIgZm9udC1mYW1pbHk9IkFyaWFsIiBmb250LXNpemU9IjE2IiBmaWxsPSJ3aGl0ZSIgdGV4dC1hbmNob3I9Im1pZGRsZSI+R1M8L3RleHQ+PC9zdmc+"

app.layout = html.Div([
    # Hidden div for callback state management
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Header Section with Logo and Branding
    html.Div([
        html.Div([
            html.A([
                html.Img(
                    src=logo_src,
                    style={
                        'height': '80px',
                        'width': 'auto',
                        'margin-right': '20px'
                    }
                )
            ], href='https://www.snhu.edu', target='_blank'),
            
            html.Div([
                html.H1('🐾 Grazioso Salvare Animal Rescue Dashboard 🐾', 
                        style={
                            'color': '#2E8B57', 
                            'fontFamily': 'Arial, sans-serif',
                            'margin': '0',
                            'fontSize': '28px'
                        }),
                html.H3('Developed by: Ifeoluwa Adewoyin | CS-340 Project Two', 
                        style={
                            'color': '#696969', 
                            'fontStyle': 'italic',
                            'margin': '5px 0',
                            'fontSize': '16px'
                        })
            ])
        ], style={
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'center',
            'padding': '20px',
            'backgroundColor': '#f8f9fa',
            'borderBottom': '3px solid #2E8B57'
        })
    ]),
    
    html.Hr(),
    
    # Interactive Filter Options Section
    html.Div([
        html.H4('🔍 Rescue Type Filters', 
                style={'color': '#2E8B57', 'textAlign': 'center', 'margin': '20px 0'}),
        
        html.Div([
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': '🌊 Water Rescue', 'value': 'Water Rescue'},
                    {'label': '⛰️ Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                    {'label': '🚨 Disaster/Individual Tracking', 'value': 'Disaster/Individual Tracking'},
                    {'label': '🔄 Reset (Show All)', 'value': 'Reset'}
                ],
                value='Reset',  # Default selection
                inline=True,
                style={
                    'textAlign': 'center',
                    'fontSize': '16px',
                    'margin': '20px 0'
                },
                inputStyle={'margin-right': '8px', 'margin-left': '20px'}
            )
        ])
    ], style={'backgroundColor': '#f8f9fa', 'padding': '20px', 'margin': '20px 0', 'borderRadius': '10px'}),
    
    html.Hr(),
    
    # Interactive Data Table Section
    html.Div([
        html.H4('📊 Available Animals', 
                style={'color': '#2E8B57', 'textAlign': 'center', 'margin': '20px 0'}),
        
        # Results summary
        html.Div(id='results-summary', 
                style={'textAlign': 'center', 'margin': '10px 0', 'fontSize': '14px', 'color': '#666'}),
        
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            
            # User-friendly features
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            row_selectable="single",
            row_deletable=False,
            selected_rows=[0],
            
            # Pagination
            page_action="native",
            page_current=0,
            page_size=15,
            
            # Styling
            style_cell={
                'textAlign': 'left',
                'padding': '10px',
                'fontFamily': 'Arial, sans-serif',
                'fontSize': '12px',
                'whiteSpace': 'normal',
                'height': 'auto',
                'maxWidth': '150px',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis'
            },
            style_header={
                'backgroundColor': '#2E8B57',
                'color': 'white',
                'fontWeight': 'bold',
                'textAlign': 'center'
            },
            style_data={
                'backgroundColor': '#F8F8F8',
                'border': '1px solid #ddd'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': '#ffffff'
                }
            ],
            
            # Fixed table height with scrolling
            fixed_rows={'headers': True},
            style_table={'height': '400px', 'overflowY': 'auto'}
        )
    ]),
    
    html.Br(),
    html.Hr(),
    
    # Charts Section
    html.Div([
        html.H4('📈 Data Visualizations', 
                style={'color': '#2E8B57', 'textAlign': 'center', 'margin': '20px 0'}),
        
        html.P('Click on any row in the table above to see the animal\'s location on the map.',
               style={'textAlign': 'center', 'fontStyle': 'italic', 'color': '#696969', 'margin': '10px 0'}),
        
        # Charts container
        html.Div([
            # Geolocation chart
            html.Div([
                html.H5('🗺️ Animal Location Map', 
                        style={'color': '#2E8B57', 'textAlign': 'center'}),
                html.Div(id='map-id', style={'textAlign': 'center'})
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '10px'}),
            
            # Breed distribution chart
            html.Div([
                html.H5('🐕 Breed Distribution', 
                        style={'color': '#2E8B57', 'textAlign': 'center'}),
                dcc.Graph(id='breed-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '10px'})
        ])
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update data table based on filter selection
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns'),
     Output('results-summary', 'children')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """
    Update the data table based on the selected filter type
    """
    # Get filtered data
    filtered_df = get_rescue_data(filter_type)
    
    # Create summary message
    if filter_type == 'Reset':
        summary = f"Showing all {len(filtered_df)} animals in the database"
    else:
        summary = f"Found {len(filtered_df)} animals suitable for {filter_type}"
    
    # Prepare data and columns for the table
    if filtered_df.empty:
        data = []
        columns = []
        summary = f"No animals found matching {filter_type} criteria"
    else:
        data = filtered_df.to_dict('records')
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns]
    
    return data, columns, summary

# Callback to highlight selected row in data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'derived_virtual_data')]
)
def update_row_styles(selected_rows, rows):
    """
    Highlight the selected row in the data table
    """
    style_data_conditional = [
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': '#ffffff'
        }
    ]
    
    if selected_rows:
        style_data_conditional.append({
            'if': {'row_index': selected_rows[0]},
            'backgroundColor': '#D2F3FF',
            'border': '2px solid #2E8B57'
        })
    
    return style_data_conditional

# Callback to update the geolocation map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    """
    Update the geolocation map based on selected animal
    """
    if not viewData:
        return html.Div("No data available for mapping", 
                       style={'textAlign': 'center', 'color': 'red', 'fontSize': '16px'})
    
    # Convert to DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Determine which row to display
    if selected_rows is None or len(selected_rows) == 0:
        row = 0  # Default to first row
    else:
        row = selected_rows[0]
    
    # Check if we have valid data
    if dff.empty or row >= len(dff):
        return html.Div("No data available for mapping", 
                       style={'textAlign': 'center', 'color': 'red', 'fontSize': '16px'})
    
    # Get coordinates
    lat_col = 'location_lat' if 'location_lat' in dff.columns else None
    lng_col = 'location_long' if 'location_long' in dff.columns else None
    
    if lat_col is None or lng_col is None:
        return html.Div("Location coordinates not available", 
                       style={'textAlign': 'center', 'color': 'red', 'fontSize': '16px'})
    
    # Extract coordinates with error handling
    try:
        lat = float(dff.iloc[row][lat_col])
        lng = float(dff.iloc[row][lng_col])
        
        if pd.isna(lat) or pd.isna(lng):
            lat, lng = 30.75, -97.48  # Default to Austin, TX
            location_note = "(Default Austin location - coordinates not available)"
        else:
            location_note = ""
            
    except (ValueError, TypeError):
        lat, lng = 30.75, -97.48  # Default to Austin, TX
        location_note = "(Default Austin location - invalid coordinates)"
    
    # Get animal information
    animal_name = str(dff.iloc[row].get('name', 'Unknown'))
    animal_breed = str(dff.iloc[row].get('breed', 'Unknown'))
    animal_type = str(dff.iloc[row].get('animal_type', 'Unknown'))
    animal_id = str(dff.iloc[row].get('animal_id', 'Unknown'))
    animal_age = str(dff.iloc[row].get('age_upon_outcome', 'Unknown'))
    
    # Create the map
    return [
        dl.Map(
            style={'width': '100%', 'height': '400px'},
            center=[lat, lng],
            zoom=12,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lng],
                    children=[
                        dl.Tooltip(f"{animal_name} - {animal_breed} {location_note}"),
                        dl.Popup([
                            html.H4(f"🐾 {animal_name}", style={'color': '#2E8B57', 'margin': '0'}),
                            html.P(f"ID: {animal_id}", style={'margin': '5px 0'}),
                            html.P(f"Type: {animal_type}", style={'margin': '5px 0'}),
                            html.P(f"Breed: {animal_breed}", style={'margin': '5px 0'}),
                            html.P(f"Age: {animal_age}", style={'margin': '5px 0'}),
                            html.P(f"Coordinates: {lat:.4f}, {lng:.4f}", 
                                   style={'margin': '5px 0', 'fontSize': '12px', 'color': '#666'})
                        ])
                    ]
                )
            ]
        )
    ]

# Callback to update the breed distribution chart
@app.callback(
    Output('breed-chart', 'figure'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_breed_chart(viewData):
    """
    Update the breed distribution pie chart based on filtered data
    """
    if not viewData:
        # Empty chart when no data
        fig = go.Figure()
        fig.update_layout(
            title="No data available",
            height=400
        )
        return fig
    
    # Convert to DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Count breeds
    if 'breed' in dff.columns and not dff.empty:
        breed_counts = dff['breed'].value_counts().head(10)  # Top 10 breeds
        
        # Create pie chart
        fig = px.pie(
            values=breed_counts.values,
            names=breed_counts.index,
            title=f"Top Breeds (Total: {len(dff)} animals)"
        )
        
        fig.update_traces(
            textposition='inside',
            textinfo='percent+label',
            hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
        )
        
        fig.update_layout(
            height=400,
            showlegend=True,
            legend=dict(
                orientation="v",
                yanchor="top",
                y=1,
                xanchor="left",
                x=1.01
            )
        )
    else:
        # Empty chart when no breed data
        fig = go.Figure()
        fig.update_layout(
            title="No breed data available",
            height=400
        )
    
    return fig

# Run the dashboard
app.run_server(debug=True, port=8050)